In [ ]:
from nav_EDA import nav_df, specs_df
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


specs_df.head()





/Users/adityachawra/Downloads/Startup/DataBase/Data/MF/investment_options-master/amfi_data_fetcher/nav_EDA.py:19: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  nav_dfexp.reset_index(inplace=True)


,Scheme Code,shceme_name,category,amc,main_type,sub_type,specific_type
0,145488,ADITYA BIRLA SUN LIFE OVERNIGHT FUND- Direct ...,Open Ended Schemes ( Debt Scheme - Overnight F...,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Debt Scheme,Overnight Fund
1,145489,ADITYA BIRLA SUN LIFE OVERNIGHT FUND- Direct ...,Open Ended Schemes ( Debt Scheme - Overnight F...,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Debt Scheme,Overnight Fund
2,145482,ADITYA BIRLA SUN LIFE OVERNIGHT FUND- Regular...,Open Ended Schemes ( Debt Scheme - Overnight F...,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Debt Scheme,Overnight Fund
3,145487,ADITYA BIRLA SUN LIFE OVERNIGHT FUND- Direct -...,Open Ended Schemes ( Debt Scheme - Overnight F...,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Debt Scheme,Overnight Fund
4,145483,ADITYA BIRLA SUN LIFE OVERNIGHT FUND- Regular ...,Open Ended Schemes ( Debt Scheme - Overnight F...,Aditya Birla Sun Life Mutual Fund,Open Ended Schemes,Debt Scheme,Overnight Fund


In [8]:

nav_equity_bm = nav_df[['date','135607']] # LIC MF Nifty 50 ETF
nav_equity_bm['date'] = nav_df['date']

nav_debt_bm = nav_df[['date','139430']] # SBI Nifty 10 yr Benchmark G-Sec ETF
nav_debt_bm['date'] = nav_df['date']

/var/folders/j3/btykbjss2l15b55qw91h85hc0000gn/T/ipykernel_30925/3523195683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nav_equity_bm['date'] = nav_df['date']
/var/folders/j3/btykbjss2l15b55qw91h85hc0000gn/T/ipykernel_30925/3523195683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nav_debt_bm['date'] = nav_df['date']


,date,139430
0,06-Apr-2020,NaN
1,07-Apr-2020,192.0797
2,08-Apr-2020,191.5776
3,09-Apr-2020,190.7810
4,10-Apr-2020,190.9685


In [ ]:
nav_pct_df = nav_df

nav_pct_df.set_index('date', inplace=True)
for col in nav_pct_df.columns:
    nav_pct_df[col] = nav_df[col].pct_change()

nav_pct_df.head(20)

In [ ]:
nav_pct_df = nav_pct_df.replace([np.inf, -np.inf], np.nan)
nav_pct_df.head(20)

In [15]:
specs_equity_df = specs_df[specs_df['sub_type'] == "Equity Scheme"]
specs_equity_df.head()

,shceme_name,category,amc,main_type,sub_type,specific_type
119354,Baroda BNP Paribas MULTI CAP FUND - Direct Pla...,Open Ended Schemes ( Equity Scheme - Multi Cap...,Baroda BNP Paribas Mutual Fund,Open Ended Schemes,Equity Scheme,Multi Cap Fund
119353,Baroda BNP Paribas MULTI CAP FUND - Direct Pla...,Open Ended Schemes ( Equity Scheme - Multi Cap...,Baroda BNP Paribas Mutual Fund,Open Ended Schemes,Equity Scheme,Multi Cap Fund
102020,Baroda BNP Paribas MULTI CAP FUND - Regular Pl...,Open Ended Schemes ( Equity Scheme - Multi Cap...,Baroda BNP Paribas Mutual Fund,Open Ended Schemes,Equity Scheme,Multi Cap Fund
102021,Baroda BNP Paribas MULTI CAP FUND - Regular Pl...,Open Ended Schemes ( Equity Scheme - Multi Cap...,Baroda BNP Paribas Mutual Fund,Open Ended Schemes,Equity Scheme,Multi Cap Fund
119988,BNP Paribas MULTI CAP Fund - Direct Plan - Gro...,Open Ended Schemes ( Equity Scheme - Multi Cap...,BNP Paribas Mutual Fund,Open Ended Schemes,Equity Scheme,Multi Cap Fund


In [ ]:
n = len(nav_pct_df['100033']) - nav_pct_df.isnull().sum()
n

In [ ]:
# sharpe ratio

rf = 0.05792/252  # risk-free rate per day
nav_quant_data = []
for columns in nav_pct_df.columns:
    sum = nav_pct_df[columns].sum() # sum of daily returns
    n = len(nav_pct_df[columns]) - nav_pct_df[columns].isnull().sum() # number of non-null days
    standard_deviation = nav_pct_df[columns].std()  # standard deviation of daily returns
    sharpe_ratio = ((sum/n - rf)/ standard_deviation) * np.sqrt(252)  # annualized Sharpe ratio
    # nav_quant_data.append([columns, sharpe_ratio, sum/n - rf, standard_deviation])

    # Sortino ratio
    downside_diff = np.minimum(nav_pct_df[columns] - rf, 0) #downside_diff
    downside_deviation = np.sqrt(np.mean(downside_diff ** 2)) # downside deviation
    sortino_ratio = ((sum/n - rf) / downside_deviation) * np.sqrt(252)  # annualized Sortino ratio

    
       
nav_quant_df = pd.DataFrame(nav_quant_data, columns=['scheme_code', 'sharpe_ratio', 'mean_daily_return', 'std_dev'])



In [ ]:
nav_quant_df.head(20)

In [ ]:
# plotting 